In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

# Tf-Idf
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords 

from nltk.stem.snowball import SnowballStemmer
import pickle
import re
import string

In [2]:
#All Files

DATA_DIR= '/Volumes/G-DRIVE mobile SSD R-Series/Avito/avito_data/'
#DATA_DIR = '/home/ubuntu/.kaggle/competitions/avito-demand-prediction/'

TRAIN_DATA = DATA_DIR + 'train.csv'
TEST_DATA = DATA_DIR + 'test.csv'
EMOJI_FILE = '/Volumes/G-DRIVE mobile SSD R-Series/Avito/avito_data/emojifile.p'

In [3]:
train = pd.read_csv(TRAIN_DATA, usecols=['title','description'])
test = pd.read_csv(TEST_DATA,usecols=['title','description'])

train['title'] = train['title'].astype('str')
train['description'] = train['description'].astype('str')

In [4]:
f = open(EMOJI_FILE,'rb')  
emoji = pickle.load(f)  
del f 
gc.collect()

stemmer = SnowballStemmer("russian") 
regex = re.compile('[%s]' % re.escape(string.punctuation + ''.join(emoji)))

def stemming_tokenizer(str_input):
    #words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = regex.sub('', str_input).lower().split()
    #words = str_input.lower().split()
    words = [stemmer.stem(word) for word in words]
    return words

In [5]:
count_vectorizer_title = CountVectorizer(stop_words=stopwords.words('russian'), tokenizer=stemming_tokenizer,
                                         lowercase=True,min_df=25)

title_counts = count_vectorizer_title.fit_transform(train['title'].append(test['title']))
train_title_counts = title_counts[:len(train)]
test_title_counts = title_counts[len(train):]

pickle.dump(train_title_counts,open('train_title_new1.p','wb'))
pickle.dump(test_title_counts,open('test_title_new1.p','wb'))


count_vectorizer_desc = TfidfVectorizer(stop_words=stopwords.words('russian'), tokenizer=stemming_tokenizer,
                                        lowercase=True, ngram_range=(1, 2),
                                        max_features=15000)
desc_counts = count_vectorizer_desc.fit_transform(train['description'].append(test['description']))
train_desc_counts = desc_counts[:len(train)]
test_desc_counts = desc_counts[len(train):]

pickle.dump(train_desc_counts,open('train_desc_new1.p','wb'))
pickle.dump(test_desc_counts,open('test_desc_new1.p','wb'))
pickle.dump(count_vectorizer_desc.get_feature_names(),open('desc_feature_names_new1.p','wb')) 
pickle.dump(count_vectorizer_title.get_feature_names(),open('title_feature_names_new1.p','wb'))


In [6]:
train_desc_counts.shape, test_desc_counts.shape, train_title_counts.shape,test_title_counts.shape

((1503424, 15000), (508438, 15000), (1503424, 13480), (508438, 13480))